# Threat Hunting Hugging Face Dataset for Gretel Processing


# INITIAL SETUP

In [3]:
#VIRTUALENVGOODNESS - YEAH BRAHHHHHH
VIRTUALENV=True
#################
VENV_NAME="venv"
%pip install jupyter ipykernel
if VIRTUALENV == True:
    try:
        !source $VENV_NAME/bin/activate
    except Exception as e:
        !pip3 install virtualenv
        !virtualenv $VENV_NAME
        !source $VENV_NAME/bin/activate
        %python3 -m ipykernel install  --user --name=



  Using cached jupyter-1.0.0-py2.py3-none-any.whl.metadata (995 bytes)
  Using cached notebook-7.1.2-py3-none-any.whl.metadata (10 kB)
  Using cached qtconsole-5.5.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.3-py3-none-any.whl.metadata (8.2 kB)
  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.10-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.10-py3-none-any.whl.metadata (4.1 kB)
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached jupyterlab_pygments-0.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached mistune-3.0.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached nbclient-0.10.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached pandocfilters-1.5.1-py

In [1]:
%pip install -q torch
%pip install numpy
%pip install pandas
%pip install -q wandb #comet_ml
%pip install pyarrow python-dotenv datasets gretel-client




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 2.9 MB/s eta 0:00:0000:0100:01
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached gretel_client-0.17.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached backports.cached_property-1.0.0.post2-py3-none-any.whl.metadata (3.8 kB)
  Using cached click-8.1.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached docker-6.1.2-py3-none-any.whl.met

### Constants


In [58]:
RUNS=0
PROJECT_NAME="Cognitive Synthesis"
DEBUG = True
LOG_LEVEL = DEBUG
PARALLEL = True


In [11]:
if RUNS == 0:
  try:
    %pprint
    DEFAULT_GH_PROJECT="synavatelabs_project"
    GH_USER = 'snyata' #this is your git user name
    NOTEBOOK_ERRORS = []
    RUNS+=1
    print("##### INITIAL SETUP COMPLETE ######")
  except Exception as e:
    print(e)
else:
  pass

##### INITIAL SETUP COMPLETE ######


In [ ]:
# Parallel Processing
import multiprocessing
import pandas as pd

if PARALLEL = True:
    # Load the data
    df = pd.read_csv("data.csv")
# Define a function to be applied to each row
def process_row(row):
    # Process the row here
    return row

# Set up multiprocessing
pool = multiprocessing.Pool()

# Apply the function to each row in parallel
result = pool.map(process_row, df.itertuples(index=False))

# Convert the result back to a DataFrame
result_df = pd.DataFrame(result, columns=df.columns)

# Close the multiprocessing pool
pool.close()

In [54]:
import os
import random
from dotenv import load_dotenv
import numpy as np
import torch
from tqdm.auto import tqdm

# SET TO FALSE IF NOT ON GOOGLE COLAB
COLAB = False

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("mps" if COLAB == False & torch.backends.mps.is_available() else "cpu")
print(f"Environment Colab == {COLAB}\n\n Your device is {device}")
if COLAB == False:
    print("FOR GPU ON COLAB SET COLAB TO TRUE")

Environment Colab == False

 Your device is mps
FOR GPU ON COLAB SET COLAB TO TRUE


# Data PreProcessing
### Logging and safe execution decorator

In [40]:
import sys
from typing import Callable, Any
import logging
from functools import wraps

# Setup basic configuration for logging
logging.basicConfig(filename='safe_apply_errors.log', level=f'logging.{LOG_LEVEL}',
                    format='%(asctime)s:%(levelname)s:%(message)s')

def safe_apply(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    A decorator to safely apply a function to a pandas DataFrame or Series column.

    Args:
        func: A function to be applied to a pandas DataFrame or Series column.

    Returns:
        The wrapped function with added exception handling.
    """
    @wraps(func)
    def wrapper(*args, **kwargs) -> Any:
        count_errors = 0
        try:
            # Assuming the first argument is always the DataFrame or Series
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            count_errors += 1
            if count_errors > 2:
                logging.error(__name__)
                sys.exit()
            # Log the exception
            logging.error(f"Error applying function {func.__name__}: {e}")
            # Optionally, return None or a default value instead of raising
            return None

    return wrapper

In [13]:
#Hugging Face Data
from datasets import load_dataset
import pandas as pd
load_dotenv()

DATASET = "Olec/cyber-threat-intelligence_v2"
ds = load_dataset(DATASET)
#ds = ds.with_format("torch", device=device)





In [14]:
# Creating a Dataframe from the training dataset


df = pd.DataFrame({
    "id": ds['train']["id"],
    "text": ds["train"]["text"],
    "entities": ds["train"]["entities"]
})

In [18]:
def data_cleaning(df):
    # Remove N/A and duplicate text, id
    print(f"NA: {df.isna().sum()}\n\n")
    print(f"NULL: {df.isnull().sum()}\n\n")
    df = df.dropna()
    df['text'] = df['text'].drop_duplicates()
    df['id'] = df['id'].drop_duplicates()
    print(df['text'].head())
    print("######  SHAPE OF YOUR DATA IS  ######")
    print(df.shape)
    return df




NA: id           0
text        11
entities     0
dtype: int64


NULL: id           0
text        11
entities     0
dtype: int64


0    A cybersquatting domain save-russia[.]today is...
1    Like the Android Maikspy, it first sends a not...
2    While analyzing the technical details of this ...
3    (Note that Flash has been declared end-of-life...
4    Figure 21. Connection of Maikspy variants to 1...
Name: text, dtype: object
######  SHAPE OF YOUR DATA IS  ######
(332, 3)


# NLP Functions

In [20]:
%pip install tqdm ipywidgets IProgress
%pip install nltk
%pip install --upgrade pip


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.


In [22]:
#Process Text NLP

import IProgress
from tqdm.notebook import tqdm
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

tqdm.pandas()

[nltk_data] Downloading package punkt to /Users/nullzero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nullzero/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nullzero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag

In [46]:
#Label Variable For Operations on Entities
# Helpful to work out exactly where in the data structure the item is


label_variable = df['entities'][x][2][0]['label']
label_variable

'threat-actor'

# TEXT FUNCTIONS

In [41]:
@safe_apply
def reduce_text(text):
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(text) for words in text if words not in stopwords.words('english')]
  type(words)
  return words

In [47]:
# TOKENIZE TEXT FUNCTION
@safe_apply
def tokenize_text(text):
  for word in text:  
    tokens = word_tokenize(word)
    tokens = [word for word in tokens if word.isalpha()]
  return ' '.join(tokens)


In [48]:
@safe_apply
def extract_features(entities):
    # Define the features you're interested in.
    features_of_interest = [
        "attack-pattern",
        "malware",
        "ip",
        "threat-actor",
        "campaign",
        "intrusion-set"
    ]
    
    # Initialize an empty list to hold labels (features) found in entities.
    features_found = []
    
    # Iterate through each entity dictionary in the entities list.
    for entity in entities:
        # If the entity's label is one of the features of interest, append it to the list.
        if entity['label'] in features_of_interest:
            features_found.append(entity['label'])
    
    # Return the list with labels found in entities.
    return features_found



In [49]:
# Isolate attack patterns
@safe_apply
def isolate_attack_patterns(entities):
    # Directly check 'label' in each entity dictionary
    attack_patterns = [entities[0][2]['label'] for entity in entities if entity == 'attack-pattern']
    return ', '.join(attack_patterns)



In [50]:

from typing import List, Dict, Any 
@safe_apply
def isolate_labels(data: pd.Series) -> pd.Series:
    # Pulls the labels out of the entities dictionaries into it's own column
    print(data)
    data["raw_labels"] = data['entities'][0][2]['labels']
    return data["raw_labels"]



In [51]:
#  ONE HOT ENCODING FUNCTION
from sklearn.preprocessing import OneHotEncoder

@safe_apply
def one_hot(data):
    one_hot = OneHotEncoder()
    #labels_array = np.array(df_labels).reshape(-1, 1)
    one_hot_encoded = one_hot.fit_transform(data)
    return one_hot_encoded
    



In [52]:
#Classify
@safe_apply
def classify_text(text):
    return nltk.classify(text)

print ("### CLASSIFY NOT IMPLEMENTED ###")

### CLASSIFY NOT IMPLEMENTED ###


# Text Processing Procedure
PROGRESS_APPLY FOR TRACKING OF PANDAS EXECUTIOn - SAFE_APPLY WILL AUTOMATICALLY LOG INCLUDING ERRORS

In [55]:
'''

ALL EXECUTION OF PROGRESS APPLYING TEXT PROCESSING ARE IN THIS STEP FOR THE ORION LIBRARY

'''

def run_all_apply():
    df = data_cleaning(df)
    #Reduce text down using NLTK and apply from Pandas
    #df["text"] = df["text"].progress_apply(reduce_text)
    print("Text reduced")

    # ATTACK PATTERNS
    df['attack_patterns'] = df['entities'].progress_apply(isolate_attack_patterns)
    print("Attack patterns done")


    # FEATURE EXTRACTION
    # Apply the function to each row's 'entities' column and store the result in a new column.
    df['features_found'] = df['entities'].progress_apply(extract_features)
    print("features extracted")

    # THREAT LABELS

    df['threat_labels'] = df.progress_apply(isolate_labels)
    print("Threat Labels extracted")

    # ONE HOT ENCODING
    #OneHot Encoding
    df['one_hot_labels']= df['threat_labels'].progress_apply(one_hot)
    print("One Hot Encoded")

    #Tokenize text
    print(df["text"])
    df["tokens"] = df["text"].progress_apply(tokenize_text)
    print("Text tokenized")









  0%|          | 0/332 [00:00<?, ?it/s]

Text reduced


  0%|          | 0/332 [00:00<?, ?it/s]

Attack patterns done


  0%|          | 0/332 [00:00<?, ?it/s]

...

  0%|          | 0/332 [00:00<?, ?it/s]

One Hot Encoded
0      [['A cybersquatting domain save-russia[.]today...
1      [['Like the Android Maikspy, it first sends a ...
2      [['While analyzing the technical details of th...
3      [['(Note that Flash has been declared end-of-l...
4      [['Figure 21. Connection of Maikspy variants t...
                             ...                        
327    [['    We also observed the execution of a pas...
328    [['We also observed that Mailto (AKA NetWalker...
329    [[' While several top-tier RaaS affiliate prog...
330    [[' The malware downloads OBS Studio files if ...
331    [[' The CVE-2021-32648 vulnerability lies with...
Name: text, Length: 332, dtype: object


  0%|          | 0/332 [00:00<?, ?it/s]

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x2a0452050>
Traceback (most recent call last):
  File "/Users/nullzero/Documents/repos/github.com/synavate/synavate-public/venv/lib/python3.10/site-packages/nltk/data.py", line 1160, in __del__
  File "/Users/nullzero/Documents/repos/github.com/synavate/synavate-public/venv/lib/python3.10/site-packages/nltk/data.py", line 1180, in closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


KeyboardInterrupt: 

In [56]:
df


,id,text,entities,attack_patterns,features_found,threat_labels,one_hot_labels
0,249,[['A cybersquatting domain save-russia[.]today...,"[{'end_offset': 16, 'id': 44656, 'label': 'att...",,"[attack-pattern, attack-pattern]",NaN,None
1,14309,"[['Like the Android Maikspy, it first sends a ...","[{'end_offset': 17, 'id': 48530, 'label': 'SOF...",,[malware],NaN,None
2,13996,[['While analyzing the technical details of th...,"[{'end_offset': 194, 'id': 48781, 'label': 'th...",,"[threat-actor, threat-actor, threat-actor]",NaN,None
3,13600,[['(Note that Flash has been declared end-of-l...,"[{'end_offset': 79, 'id': 51687, 'label': 'TIM...",,"[malware, malware, malware]",NaN,None
4,14364,[['Figure 21. Connection of Maikspy variants t...,"[{'end_offset': 191, 'id': 51779, 'label': 'UR...",,"[malware, malware]",NaN,None
...,...,...,...,...,...,...,...
327,1852,[[' We also observed the execution of a pas...,"[{'end_offset': 71, 'id': 46982, 'label': 'too...",,[],NaN,None
328,535,[['We also observed that Mailto (AKA NetWalker...,"[{'end_offset': 28, 'id': 1765, 'label': 'malw...",,"[malware, malware, attack-pattern]",NaN,None
329,2633,[[' While several top-tier RaaS affiliate prog...,"[{'end_offset': 98, 'id': 47813, 'label': 'thr...",,"[threat-actor, threat-actor, threat-actor, thr...",NaN,None
330,3281,[[' The malware downloads OBS Studio files if ...,"[{'end_offset': 12, 'id': 48378, 'label': 'mal...",,[malware],NaN,None


In [ ]:
%env SOURCE_BUCKET=example-state-backend

In [ ]:
import os
from processed.gcp_upload import upload_blob
from datetime import datetime
# Turn processed dataframe to csv 
csv_output = df.tocsv(f"./processed/gretel-threat-intelligence-{datetime.now()}.csv")
%export DATA_FILE= csv_output
# Upload blob
upload_blob("gretel-threat-intelligence", os.getenv(DATA_FILE), "gretel-threat-intelligence.csv")


In [ ]:
!gcloud storage cp $DATA_FILE gs://$SOURCE_BUCKET